# Linear Algebra

## Vectors
Vector is a geometric object that has magnitude (or length) and direction. Vectors can be added to other vectors according to vector algebra. Concretely, vectors are points in some finite-dimensional space.

In [1]:
from typing import List

Vector = List[float]

height_weight_age = [70,  # inches,
                     170, # pounds,
                     40]  # years

grades = [95, # exam1
          80, # exam2
          75, # exam3
          62] # exam4

### Vector arithmetic

In [2]:
def add(v: Vector, w: Vector) -> Vector:
    """Adds corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    
    return [v_i + w_i for v_i, w_i in zip(v, w)]


add([1,2,3],[4,5,6])

[5, 7, 9]

In [3]:
def subtract(v: Vector, w: Vector) -> Vector:
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    
    return [v_i - w_i for v_i, w_i in zip(v, w)]


subtract([1,2,3],[4,5,6])

[-3, -3, -3]

In [4]:
def vector_sum(vectors: List[Vector]) -> Vector:
    """Sums all corresponding elements"""
    assert vectors, "no vectors provided!"
    
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"
    
    return [sum(vector[i] for vector in vectors) for i in range(num_elements)]


vector_sum([[1,2],[3,4],[5,6],[7,8]])

[16, 20]

In [5]:
def scalar_multiply(c: float, v: Vector) -> Vector:
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]


scalar_multiply(2,[1,2,3])

[2, 4, 6]

In [6]:
def vector_mean(vectors: List[Vector]) -> Vector:
    """Computes the element-wise average"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))


vector_mean([[1,2],[3,4],[5,6]])

[3.0, 4.0]

In [7]:
def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectos must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))


dot([1, 2, 3], [4, 5, 6]) # 1*4 + 2*5 + 3*6

32

If w has magnitude 1, the dot product measures how far the vector v extends in the w direction. For example, if w = [1, 0], then dot(v,w) is just the first component of v. Another way of saying this is that it's the length of the vector you'd get if you projected v onto w.

In [8]:
def sum_of_squares(v: Vector) -> float:
    """Returns v_1 * v_1 + ... + v_n + v_n"""
    return dot(v,v)


sum_of_squares([1,2,3])

14

In [9]:
import math

def magnitude(v: Vector) -> float:
    """Returns the magnitude (or length) of v"""
    return math.sqrt(sum_of_squares(v)) # math.sqrt is square root funtion


magnitude([3,4])

5.0

To compute the distance between two vectors: $$\sqrt{(v_1 - w_1)^2 + ... + (v_n - w_n)^2}$$

In [10]:
def squared_distance(v: Vector, w: Vector) -> Vector:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v,w))

def distance(v: Vector, w: Vector) -> Vector:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v,w))

#alternative
def distance(v: Vector, w: Vector) -> Vector:
    return magnitude(subtract(v,w))

distance([3,4],[9,12])

10.0

## Matrices

In [11]:
Matrix = List[List[float]]

A = [[1,2,3],
     [4,5,6]]

B = [[1,2],
     [3,4],
     [5,6]]

In [12]:
from typing import Tuple

def shape(A: Matrix) -> Tuple[int,int]:
    """Returns (# of rows of A, # of columns of A)"""
    num_rows = len(A)
    num_columns = len(A[0]) if A else 0
    return num_rows, num_columns

shape(A)

(2, 3)

In [13]:
def get_row(A: Matrix, i: int) -> Vector:
    """Returns the i-th row of A (as a Vector)"""
    return A[i]

def get_column(A: Matrix, j: int) -> Vector:
    """Returns the j-th column of A (as a Vector)"""
    return [A_i[j] for A_i in A]

from typing import Callable

def make_matrix(num_rows: int,
                num_cols: int,
                entry_fn: Callable[[int, int], float]) -> Matrix:
    """Returns a num_rows x num_cols matrix whose (i,j)-th entry is entry_fn(i,j)"""
    return [[entry_fn(i,j) for j in range(num_cols)] for i in range(num_rows)]

def identity_matrix(n: int) -> Matrix:
    """Returns the n x n identity matrix"""
    return make_matrix(n,n, lambda i, j: 1 if i==j else 0)

identity_matrix(5)

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]

# Gradient Descent

In machine learning we frequently will need to maximize or minimize functions that take as input a vector of real numbers and output a single real number. For functions like this, the gradient (if you remember your calculus, this is the vector of partial derivatives) gives the direction in which the function most quickly increases. Accordingly, one approach to maximize a function is to take a pick a random starting point, compute the gradient, take a small step in the direction of the gradient (i.e., the direction that causes the function to increase the most), and repeat with the new starting point. Similarly, you can try to minimize a function by taking small steps in the opposite direction.

If a function has a unique global minimum, this procedure is likely to find it. If a function has multiple (local) minima, this procedure might "find" the wrong one of them, in which case you might rerun the procedure from different starting points. If a function has no minimum, then it's possible the procedure might go on forever.

Wikipedia: In numerical mathematics, numerical differentiation is the approximate calculation of the derivative from given function values, usually by means of a difference quotient. This is necessary if the derivative function is not given or the function itself is only available indirectly, for example via measured values. 

In [14]:
from typing import Callable

def difference_quotient(f: Callable[[float], float],
                        x: float,
                        h: float) -> float:
    return (f(x + h) - f(x)) / h

When f is a function of many variables, it has multiple partial derivatives, each indicating how f changes when we make small changes in just one of the input variables. We calculate its ith partial derivative by treating it as a function of just its ith variable, holding the other variables fixed.

In [15]:
def partial_difference_quotient(f: Callable[[Vector], float],
                                v: Vector,
                                i: int,
                                h: float) -> float:
    """Returns the i-th partial difference quotient of f at v"""
    w = [v_j + (h if j == i else 0) for j, v_j in enumerate(v)]
    return (f(w) - f(v)) / h

def estimate_gradient(f: Callable[[Vector], float],
                      v: Vector,
                      h: float = 0.0001):
    return [partial_difference_quotient(f, v, i, h) for i in range(len(v))]

## Using the Gradient

To compute the minimum for the sum_of_squares function (obviously it is 0).

In [16]:
import random

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """Moves 'step_size' in the 'gradient' direction from 'v'"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

# to compute the gradient directly
def sum_of_squares_gradient(v: Vector) -> Vector:
    return [2 * v_i for v_i in v]

In [17]:
# pick a random starting point
v = [random.uniform(-10, 10) for i in range(3)]

for epoch in range(1000):
    grad = estimate_gradient(sum_of_squares, v)
    v = gradient_step(v, grad, -0.01)
    if epoch % 100 == 0:
        print(epoch, v)

0 [1.3275126616204336, 2.0471852377664312, -9.471442186692585]
100 [0.17601077060707127, 0.2714534280450831, -1.2561418255044021]
200 [0.02329910120839225, 0.03595666405124445, -0.16663234006143507]
300 [0.0030465474327986963, 0.004725187795727537, -0.022142075958856148]
400 [0.0003606627453448042, 0.0005832832847834357, -0.002979841302456668]
500 [4.461910910148e-06, 3.3985747983549746e-05, -0.0004385542523739083]
600 [-4.2777285561916555e-05, -3.8861847400932966e-05, -0.0001015298923908456]
700 [-4.904212681886677e-05, -4.852286314882374e-05, -5.683387144417837e-05]
800 [-4.987296728411462e-05, -4.9804102766801246e-05, -5.090630499596878e-05]
900 [-4.998315297763518e-05, -4.9974020195932166e-05, -5.012019376607058e-05]


After a certain point we stop improving our prediction. The further increase the prediction we can either reduce the stepsize or reduce h since we can no longer optimaly predict the gradient.

## Using Gradient Descent to Fit Models

Example: In this case we know the parameters of the linear relationship between x and y, but imagine we'd like to learn them from the data. We'll use gradient descent to find the slope and intercept that minimize the average squared error.

In [18]:
# ranges from - 50 to 49, y is always 20 * x + 5
inputs = [(x, 20 * x + 5) for x in range(-50, 50)]

def linear_gradient(x: float, y: float, theta: Vector) -> Vector:
    slope, intercept = theta
    predicted = slope * x + intercept
    error = (predicted - y)
    squared_error = error ** 2
    grad = [2 * error * x, 2 * error]
    return grad

# random values for slope and intercept
theta = [random.uniform(-1,1), random.uniform(-1,1)]

learning_rate = 0.001

for epoch in range(4000):
    grad = vector_mean([linear_gradient(x,y,theta) for x,y in inputs])
    theta = gradient_step(theta,grad,-learning_rate)
    if epoch % 200 == 0:
        print(epoch, theta)
        
theta

0 [33.01363393477094, 0.6849026993548983]
200 [19.99826639873727, 2.1135528563548363]
400 [19.998838257856573, 3.0656986334551553]
600 [19.99922147910432, 3.7037622411154745]
800 [19.99947828802765, 4.131349252697213]
1000 [19.9996503839735, 4.417889105900587]
1200 [19.99976571101975, 4.609908707174461]
1400 [19.999842995394648, 4.7385872377913]
1600 [19.999894786147962, 4.824818873165302]
1800 [19.999929492802867, 4.882605474423722]
2000 [19.999952750852177, 4.921330140499186]
2200 [19.99996833682148, 4.947280788746358]
2400 [19.99997878148241, 4.964671155473743]
2600 [19.999985780786716, 4.9763250013442875]
2800 [19.999990471246374, 4.9841346194911225]
3000 [19.999993614474736, 4.989368096600474]
3200 [19.99999572085348, 4.99287521847751]
3400 [19.999997132405845, 4.995225454009914]
3600 [19.99999807833263, 4.9968004226740845]
3800 [19.999998712228688, 4.997855859994694]


[19.999999135294, 4.9985602639912114]

# Minibatch and Stochastic Gradient Descent

In [19]:
from typing import TypeVar, Iterator

T = TypeVar('T')

def minibatches(dataset: List[T],
                batch_size: int,
                shuffle: bool = True) -> Iterator[List[T]]:
    """Generates 'batch_size'-sized minibatches from the dataset"""
    batch_starts = [start for start in range(0,len(dataset),batch_size)]
    
    if shuffle: random.shuffle(batch_starts)
        
    for start in batch_starts:
        end = start + batch_size
        yield dataset[start:end]

In [20]:
theta = [random.uniform(-1, 1), random.uniform(-1, 1)]

for epoch in range(1000):
    for batch in minibatches(inputs, batch_size=20):
        grad = vector_mean([linear_gradient(x, y, theta) for x, y in batch])
        theta = gradient_step(theta, grad, -learning_rate)
    if epoch%100 == 0: print(epoch, theta)

theta

0 [18.203900463218186, 2.811606844794329]
100 [20.066553626911066, 4.194745136887922]
200 [19.980792220761987, 4.812616296384315]
300 [19.998916893683376, 4.969689299743435]
400 [20.00002304142001, 4.995348010220143]
500 [20.00011692020163, 4.99922104140855]
600 [19.999988356428258, 4.9998100075877865]
700 [19.99999837165458, 4.999954736021836]
800 [20.000000466918248, 4.9999913886067615]
900 [19.999999625638402, 4.9999976921368585]


[19.999999971038772, 4.999999355730051]

In [21]:
theta = [random.uniform(-1, 1), random.uniform(-1, 1)]

for epoch in range(100):
    for x, y in inputs:
        grad = linear_gradient(x, y, theta)
        theta = gradient_step(theta, grad, -learning_rate)
    if epoch%10 == 0: print(epoch, theta)

0 [20.11814367851677, -0.8802633514541575]
10 [20.07620159805196, 1.2072618226666982]
20 [20.04914961855205, 2.5537043050753008]
30 [20.031701236915577, 3.422152454694023]
40 [20.02044712639024, 3.9822968324655155]
50 [20.01318828939189, 4.3435869383461725]
60 [20.008506374841414, 4.576617110795876]
70 [20.00548656486774, 4.726920316536092]
80 [20.003538798249018, 4.823865075401264]
90 [20.002282508168367, 4.886393922132545]


# k-Nearest Neighbors